In [ ]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta


import yfinance as yf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from sklearn.preprocessing import MinMaxScaler



In [ ]:
# Get the current date
today = datetime.today()
day_ago = today - relativedelta(days=1)
final_date = day_ago.strftime("%Y-%m-%d")
final_date


'2024-07-10'

In [ ]:
# time period
delta = 10
years_ago = today - relativedelta(years=delta, days=1)

# Format the date as YYYY-MM-DD
start_date = years_ago.strftime("%Y-%m-%d")

start_date

'2014-07-10'

In [ ]:
ticker = 'SONY'
stock_data = yf.download(ticker, start=start_date, end=final_date)
stock_data


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-07-10,16.570000,16.700001,16.430000,16.650000,16.650000,1592200
2014-07-11,16.620001,16.680000,16.530001,16.660000,16.660000,1072500
2014-07-14,16.719999,16.750000,16.650000,16.670000,16.670000,1087400
2014-07-15,16.600000,16.600000,16.430000,16.530001,16.530001,1477600
2014-07-16,16.709999,16.809999,16.700001,16.760000,16.760000,2187500
...,...,...,...,...,...,...
2024-07-02,85.910004,86.669998,85.620003,86.660004,86.660004,818600
2024-07-03,85.019997,85.900002,85.019997,85.650002,85.650002,531400
2024-07-05,86.070000,86.889999,86.070000,86.820000,86.820000,608800


In [ ]:
df = stock_data.reset_index()[['Date', 'Close']]
df

,Date,Close
0,2014-07-10,16.650000
1,2014-07-11,16.660000
2,2014-07-14,16.670000
3,2014-07-15,16.530001
4,2014-07-16,16.760000
...,...,...
2511,2024-07-02,86.660004
2512,2024-07-03,85.650002
2513,2024-07-05,86.820000
2514,2024-07-08,86.599998


In [ ]:
data = df.tail(2516)
data

,Date,Close
0,2014-07-10,16.650000
1,2014-07-11,16.660000
2,2014-07-14,16.670000
3,2014-07-15,16.530001
4,2014-07-16,16.760000
...,...,...
2511,2024-07-02,86.660004
2512,2024-07-03,85.650002
2513,2024-07-05,86.820000
2514,2024-07-08,86.599998


In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))
scaled_data

array([[-0.99785831],
       [-0.99767983],
       [-0.99750135],
       ...,
       [ 0.25450655],
       [ 0.25058006],
       [ 0.31911484]])

In [ ]:
len(scaled_data)

2516

In [ ]:
type(scaled_data)

numpy.ndarray

In [ ]:
def create_dataset(data, time_step):
  X, y = [], []
  for i in range(len(data) - time_step - 1):
    X.append(data[i:(i + time_step), 0])
    y.append(data[(i + time_step), 0])
  return np.array(X), np.array(y)

In [ ]:
time_step = 90
"""
time_step is used to define the length of the sequence of past data
points that will be used as input for predicting the next data point.
This is common in time series analysis and forecasting where the goal
is to use historical data to predict future values.
"""
X, y = create_dataset(scaled_data, time_step)

In [ ]:
train_size = 0.8
X_train, X_test = X[:int(X.shape[0]*train_size)], X[int(X.shape[0]*train_size):]
y_train, y_test = y[:int(y.shape[0]*train_size)], y[int(y.shape[0]*train_size):]
print(len(X_train), len(X_test), len(y_train), len(y_test))

1940 485 1940 485


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=90, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(units=90, return_sequences=False))
model.add(Dense(units=90))
model.add(Dense(units=1))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 90, 90)            33120     
                                                                 
 lstm_1 (LSTM)               (None, 90)                65160     
                                                                 
 dense (Dense)               (None, 90)                8190      
                                                                 
 dense_1 (Dense)             (None, 1)                 91        
                                                                 
Total params: 106561 (416.25 KB)
Trainable params: 106561 (416.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=4)

Epoch 1/3
485/485 [==============================] - 31s 56ms/step - loss: 0.0045
Epoch 2/3
485/485 [==============================] - 27s 56ms/step - loss: 0.0018
Epoch 3/3
485/485 [==============================] - 27s 55ms/step - loss: 0.0012


In [ ]:
X_train

array([[-0.99785831, -0.99767983, -0.99750135, ..., -0.92539713,
        -0.92682494, -0.9307514 ],
       [-0.99767983, -0.99750135, -1.        , ..., -0.92682494,
        -0.9307514 , -0.92664646],
       [-0.99750135, -1.        , -0.99589506, ..., -0.9307514 ,
        -0.92664646, -0.93342853],
       ...,
       [ 0.59896494,  0.56915937,  0.59200439, ...,  0.23076923,
         0.22684274,  0.251651  ],
       [ 0.56915937,  0.59200439,  0.56380518, ...,  0.22684274,
         0.251651  ,  0.26771372],
       [ 0.59200439,  0.56380518,  0.62127442, ...,  0.251651  ,
         0.26771372,  0.2289845 ]])

In [ ]:
scaled_data

array([[-0.99785831],
       [-0.99767983],
       [-0.99750135],
       ...,
       [ 0.25450655],
       [ 0.25058006],
       [ 0.31911484]])

In [ ]:
last_fifty = scaled_data[-90:]
last_fifty

array([[0.23648045],
       [0.27592359],
       [0.24843843],
       [0.23380342],
       [0.25361411],
       [0.25593438],
       [0.26146709],
       [0.25343573],
       [0.2711048 ],
       [0.27271102],
       [0.25950385],
       [0.26450115],
       [0.29394969],
       [0.29341428],
       [0.30447983],
       [0.28841698],
       [0.28663225],
       [0.23915761],
       [0.25272181],
       [0.24950924],
       [0.23523112],
       [0.24290559],
       [0.22791369],
       [0.23576653],
       [0.21399259],
       [0.21881138],
       [0.20988759],
       [0.21559881],
       [0.20649651],
       [0.20524718],
       [0.20399785],
       [0.17383553],
       [0.18579338],
       [0.16098525],
       [0.16633944],
       [0.15598795],
       [0.16062822],
       [0.16509012],
       [0.18115297],
       [0.17633418],
       [0.17437094],
       [0.18615027],
       [0.17865432],
       [0.18436554],
       [0.20560421],
       [0.21381408],
       [0.21916827],
       [0.162

In [ ]:
prediction = last_fifty.reshape(1, -1)
forecast = []
prediction

array([[0.23648045, 0.27592359, 0.24843843, 0.23380342, 0.25361411,
        0.25593438, 0.26146709, 0.25343573, 0.2711048 , 0.27271102,
        0.25950385, 0.26450115, 0.29394969, 0.29341428, 0.30447983,
        0.28841698, 0.28663225, 0.23915761, 0.25272181, 0.24950924,
        0.23523112, 0.24290559, 0.22791369, 0.23576653, 0.21399259,
        0.21881138, 0.20988759, 0.21559881, 0.20649651, 0.20524718,
        0.20399785, 0.17383553, 0.18579338, 0.16098525, 0.16633944,
        0.15598795, 0.16062822, 0.16509012, 0.18115297, 0.17633418,
        0.17437094, 0.18615027, 0.17865432, 0.18436554, 0.20560421,
        0.21381408, 0.21916827, 0.16223458, 0.1033375 , 0.08763168,
        0.05015178, 0.06407287, 0.15473863, 0.19489569, 0.19328933,
        0.19971447, 0.19685879, 0.1684812 , 0.15527403, 0.14706416,
        0.13796185, 0.1304659 , 0.10512223, 0.14688565, 0.17454932,
        0.19703731, 0.24040694, 0.25129398, 0.26182412, 0.23005531,
        0.23344639, 0.22202395, 0.20346245, 0.20

In [ ]:
final_forecast = np.array(forecast)
final_forecast

array([], dtype=float64)

In [ ]:
for _ in range(0, 90):
  temp = model.predict(prediction[-1:]) # predict next value
  prediction[-1] = np.append(prediction[-1][1:], temp) # delete first and append to prediction
  final_forecast = np.append(final_forecast, temp)


1/1 [==============================] - 0s 39ms/step


In [ ]:
final_forecast

array([0.27835003, 0.28670838, 0.29020822, 0.29184961, 0.29265192,
       0.29305932, 0.29332703, 0.29362735, 0.29408085, 0.29476807,
       0.29573646, 0.29700708, 0.29857981, 0.30043924, 0.30255944,
       0.30490822, 0.30745065, 0.31015149, 0.31297776, 0.31589934,
       0.3188906 , 0.32193026, 0.32500169, 0.32809234, 0.33119392,
       0.33430138, 0.33741254, 0.34052724, 0.34364736, 0.34677541,
       0.34991512, 0.35307038, 0.35624486, 0.35944262, 0.36266688,
       0.36592075, 0.36920676, 0.37252706, 0.37588307, 0.37927607,
       0.38270706, 0.38617635, 0.3896842 , 0.393231  , 0.3968164 ,
       0.40044054, 0.40410304, 0.40780377, 0.41154251, 0.41531935,
       0.4191339 , 0.42298624, 0.4268764 , 0.43080419, 0.43476987,
       0.4387733 , 0.44281489, 0.44689447, 0.45101237, 0.4551686 ,
       0.45936343, 0.46359685, 0.4678691 , 0.47218019, 0.47653031,
       0.48091951, 0.48534775, 0.48981538, 0.49432203, 0.49886799,
       0.50345308, 0.50807744, 0.51274091, 0.51744348, 0.52218

In [ ]:
aaa = final_forecast.reshape(-1, 1)
fff = scaler.inverse_transform(aaa)

In [ ]:
fff

array([[ 88.15594983],
       [ 88.62426869],
       [ 88.82036447],
       [ 88.9123317 ],
       [ 88.95728504],
       [ 88.98011153],
       [ 88.99511156],
       [ 89.01193838],
       [ 89.03734809],
       [ 89.07585257],
       [ 89.13011183],
       [ 89.20130478],
       [ 89.28942473],
       [ 89.39360839],
       [ 89.51240302],
       [ 89.64400519],
       [ 89.78645788],
       [ 89.93778569],
       [ 90.09614178],
       [ 90.25983797],
       [ 90.42743822],
       [ 90.59775025],
       [ 90.76984232],
       [ 90.94301142],
       [ 91.11679335],
       [ 91.29090424],
       [ 91.46522218],
       [ 91.63973883],
       [ 91.81455939],
       [ 91.98982412],
       [ 92.16574176],
       [ 92.34253104],
       [ 92.52039736],
       [ 92.69956781],
       [ 92.88022274],
       [ 93.06253747],
       [ 93.24665227],
       [ 93.43268904],
       [ 93.62072625],
       [ 93.81083572],
       [ 94.00307422],
       [ 94.19745844],
       [ 94.39400341],
       [ 94

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.save('path_to_my_model.keras')

In [ ]:
fff.reshape(-1)

array([ 88.07974143,  88.6690224 ,  89.33495493,  90.01567411,
        90.69137763,  91.3559444 ,  92.00791506,  92.6475733 ,
        93.27594382,  93.89439844,  94.50445883,  95.10767531,
        95.70553831,  96.29941593,  96.89051324,  97.4798399 ,
        98.06820846,  98.65622519,  99.24431068,  99.83271076,
       100.42152055, 101.01071198, 101.60015554, 102.18964837,
       102.77893836, 103.36773669, 103.95574884, 104.54267682,
       105.12823642, 105.7121629 , 106.29421674, 106.87418706,
       107.45188821, 108.02716546, 108.59988588, 109.1699383 ,
       109.7372276 , 110.3016747 , 110.86320855, 111.42176954,
       111.97729351, 112.52972544, 113.07900344, 113.62506793,
       114.16785013, 114.70728129, 115.24328805, 115.7757925 ,
       116.3047121 , 116.82995976, 117.35145068, 117.86909546,
       118.38280242, 118.89247988, 119.39804302, 119.89939787,
       120.39645733, 120.8891343 , 121.37735086, 121.86102447,
       122.34007723, 122.81443118, 123.28402216, 123.74